<a href="https://colab.research.google.com/github/11kartheek/Qlora_peft/blob/main/data_prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [35]:
import pandas as pd
from tqdm import tqdm

In [36]:
from datasets import load_dataset

dataset_name = "OpenAssistant/oasst1"

In [37]:
dataset = load_dataset(dataset_name, split="train")

In [73]:
df = dataset.to_pandas()

In [74]:
df['lang'].unique()

array(['en', 'es', 'de', 'ru', 'ja', 'pt-BR', 'ca', 'fr', 'pl', 'vi',
       'zh', 'hu', 'ko', 'eu', 'it', 'uk-UA', 'id', 'ar', 'fi', 'tr',
       'da', 'th', 'sv', 'cs'], dtype=object)

In [75]:
df = df[df['lang']== 'en']

In [78]:
ids = df['parent_id'].unique()

In [79]:
m_ids = df['message_id'].unique()

In [86]:
len(m_ids)

39283

In [116]:
parent = train_data.loc[train_data['msg_id'] == '123']

In [117]:
df[df['message_id'] == msg]

,message_id,parent_id,user_id,created_date,text,role,lang,review_count,review_result,deleted,rank,synthetic,model_name,detoxify,message_tree_id,tree_state,emojis,labels
1340,413a0f66-cdf4-4fda-b71b-7eaa8e1d4475,d3de5a3a-a67c-4930-84c9-2afc2f43a9f5,097aaa4d-bf30-43db-a454-2c525ec772a9,2023-02-10T10:28:48.804709+00:00,Creo que las inteligencias artificiales deberí...,assistant,en,3,False,False,NaN,False,None,"{'toxicity': 0.000328813650412485, 'severe_tox...",d3de5a3a-a67c-4930-84c9-2afc2f43a9f5,ready_for_export,"{'name': ['+1'], 'count': [1]}","{'name': ['spam', 'fails_task', 'lang_mismatch..."


In [118]:
df = df.sort_values(by=['created_date'])

In [120]:
train_data = pd.DataFrame(columns=['msg_id','input', 'output','role'])
for msg in tqdm(m_ids, desc="messages"):
  parent = df[df['message_id'] == msg]['parent_id'].unique()[0]
  role = df[df['message_id'] == msg]['role'].unique()[0]
  check_len = len(df[df['message_id'] == parent])
  if (parent is not None) and check_len > 0 :
      parent_text = df[df['message_id'] == parent]['text'].unique()[0]
      parent_role = df[df['message_id'] == parent]['role'].unique()[0]
      text = df[df['message_id'] == msg]['text'].unique()[0]
      check_len = len(train_data.loc[train_data['msg_id'] == parent])
      if check_len > 0:
        parent_df = train_data.loc[train_data['msg_id'] == parent].reset_index(drop=True)
        parent_input = parent_df['input'][0]
        parent_output = parent_df['output'][0]
        parent_prompt = parent_input + parent_output
        train_data = pd.concat([train_data,pd.DataFrame({'msg_id': msg,'input': parent_prompt, 'output': role+":\n"+text, 'role': role},index=[0])],ignore_index=True).reset_index(drop=True)
      else:
        train_data = pd.concat([train_data,pd.DataFrame({'msg_id': msg,'input': parent_role +":\n"+parent_text, 'output': role+":\n"+text, 'role': role},index=[0])],ignore_index=True).reset_index(drop=True)
  else:
      text = df[df['message_id'] == msg]['text'].unique()[0]

      train_data = pd.concat([train_data,pd.DataFrame({'msg_id': msg,'input': "", 'output': role+":\n"+text, 'role': role},index=[0])],ignore_index=True).reset_index(drop=True)

#

messages: 100%|██████████| 39283/39283 [38:25<00:00, 17.04it/s]


In [122]:
train_data.shape

(39283, 4)

In [121]:
train_data.to_csv('train_data.csv',index=False)

In [135]:
train_data.value_counts('role')

,count
role,
assistant,24083
prompter,15200


In [128]:
train_data['input'][2]

'prompter:\nCan you write a short introduction about the relevance of the term "monopsony" in economics? Please use examples related to potential monopsonies in the labour market and cite relevant research.assistant:\n"Monopsony" refers to a market structure where there is only one buyer for a particular good or service. In economics, this term is particularly relevant in the labor market, where a monopsony employer has significant power over the wages and working conditions of their employees. The presence of a monopsony can result in lower wages and reduced employment opportunities for workers, as the employer has little incentive to increase wages or provide better working conditions.\n\nRecent research has identified potential monopsonies in industries such as retail and fast food, where a few large companies control a significant portion of the market (Bivens & Mishel, 2013). In these industries, workers often face low wages, limited benefits, and reduced bargaining power, leading

In [136]:
train_df = pd.read_csv('train_data.csv')

In [138]:
train_df = train_df.loc[train_df['role'] == 'assistant']

In [139]:
train_df.isnull().sum()

,0
msg_id,0
input,81
output,0
role,0


In [140]:
train_df.dropna(inplace=True)

In [141]:
train_df['input_length'] = train_df['input'].apply(lambda x: len(x))

In [142]:
train_df['input_length'].describe()

,input_length
count,24002.000000
mean,694.669278
std,876.283353
min,12.000000
25%,101.000000
50%,335.000000
75%,1014.000000
max,12626.000000


In [145]:
print(train_df.sort_values(by = 'input_length',ascending = False).reset_index(drop=True)['input'][0])

prompter:
please generate the code to leverage azure java sdk to process ChangeFeed of Cosmos DB NoSQL.assistant:
Here is a sample code to process the Change Feed of a Cosmos DB NoSQL database using the Azure Java SDK:

import com.microsoft.azure.cosmosdb.ChangeFeedOptions;
import com.microsoft.azure.cosmosdb.ChangeFeedProcessor;
import com.microsoft.azure.cosmosdb.ChangeFeedProcessorBuilder;
import com.microsoft.azure.cosmosdb.ConnectionPolicy;
import com.microsoft.azure.cosmosdb.ConsistencyLevel;
import com.microsoft.azure.cosmosdb.DocumentClient;
import com.microsoft.azure.cosmosdb.DocumentClientException;
import com.microsoft.azure.cosmosdb.DocumentCollection;
import com.microsoft.azure.cosmosdb.FeedOptions;
import com.microsoft.azure.cosmosdb.FeedResponse;
import com.microsoft.azure.cosmosdb.ResourceResponse;
import com.microsoft.azure.cosmosdb.internal.directconnectivity.WFConstants;

import java.util.UUID;

public class CosmosDbChangeFeedProcessor {
  private static final String

In [146]:
train_df.to_csv('train_data_filtered.csv',index=False)